In [70]:
import os
import hf00
import torch
import torchaudio
import numpy as np
import pandas as pd
from IPython.display import display, Audio
import torchaudio.transforms as T
import librosa
import math
import matplotlib.pyplot as plt
from scipy.signal import convolve
from torch.utils.data import Dataset, DataLoader
import random
import torch.nn as nn
import torch.optim as optim

speech_ = []
noise_ = []

In [81]:
datas_dir = '../audioData/MUSAN/MUSAN/musan/'
noise_dir = os.listdir(datas_dir+'noise')
speech_dir = os.listdir(datas_dir+'speech')

noise_files = []
speech_files = []

In [82]:
for i in noise_dir:
    nois_dir = os.path.join((datas_dir+'noise'), i)
    if os.path.isdir(nois_dir):
        noise_files.extend(os.listdir(nois_dir))

for i in speech_dir:
    spech_dir = os.path.join((datas_dir+'speech'), i)
    if os.path.isdir(spech_dir):
        speech_files.extend(os.listdir(spech_dir))


In [93]:
noise = []
speech = []


In [94]:

for i in range(0, len(noise_files)):
    folder = noise_files[i].split('-')
    #print(folder[1])
    if folder[1] == 'free':
        fname = datas_dir+'noise/free-sound/'+noise_files[i]
        noise.append(fname)
    else:
        fname = datas_dir+'noise/sound-bible/'+noise_files[i]
        noise.append(fname)

for i in range(0, len(speech_files)):
    folder = speech_files[i].split('-')
    #print(folder[1])
    if folder[1] == 'librivox':
        fname = datas_dir+'speech/librivox/'+speech_files[i]
        speech.append(fname)
    else:
        fname = datas_dir+'speech/us-gov/'+speech_files[i]
        speech.append(fname)

len(noise_files), len(noise), len(speech), len(speech_files)

(930, 930, 426, 426)

In [196]:
SAMPLE_RATE = 16000
rir_dir = '../../RIR/MIT_IR_Survey/Audio/'
rir_files = os.listdir(rir_dir)[1:]
speech_ = speech
noise_ = noise

def get_seconds(audio):

    duration = (int)(audio.shape[0]/SAMPLE_RATE)
    audio_list = []
    
    for i in range(0, duration*SAMPLE_RATE, SAMPLE_RATE):
        audio_list.append(audio[i:i+SAMPLE_RATE])
    return audio_list

def round_up_audio(audio):
    
    rem = audio.shape[0]%SAMPLE_RATE
    zero_len = SAMPLE_RATE-rem
    added_arr = np.zeros(zero_len, audio.dtype)
    ext_audio = np.concatenate((audio, added_arr), axis=None)

    return ext_audio

def add_echo_from_file(filename, audio):

    rir_wav,sr_rir = librosa.load(filename, sr=SAMPLE_RATE)
    echo_audio = convolve(audio, rir_wav, mode='full')

    return echo_audio[0:SAMPLE_RATE]

def get_noise_from_sound(signal,noise,SNR):
    
    RMS_s=math.sqrt(np.mean(signal**2))
    #required RMS of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    
    #current RMS of noise
    RMS_n_current=math.sqrt(np.mean(noise**2))
    noise=noise*(RMS_n/RMS_n_current)
    
    return noise

def add_noise(audio, noise):

    SNR_list = [i for i in range(0,10)]
    SNR_choice = random.choice(SNR_list)
    
    noise = get_noise_from_sound(audio, noise, SNR_choice)
    noisy_audio = audio + noise
    return noisy_audio, noise

def random_sec(audio):

    duration = (int)(audio.shape[0]/SAMPLE_RATE)
    #print(audio.shape, duration)
    if duration > 1:
        sec_choice = random.choice([i for i in range(0,duration-1)])
        rand_sec = audio[(sec_choice*SAMPLE_RATE):(sec_choice+1)*SAMPLE_RATE]
    if duration == 0:
        rand_sec = np.zeros((SAMPLE_RATE,))
    else:
        rand_sec = audio[0:SAMPLE_RATE]
    return rand_sec

In [190]:
wav, sr = librosa.load(noise_[15])

rand = random_sec(wav)
rand.shape

(16000,)

In [197]:
def get_random_audio_sec(audio_filename):
    
    # Choosing a random background and echo filename
    bg1_file = random.choice(noise_)
    bg2_file = random.choice(noise_)
    rir_file = rir_dir + random.choice(rir_files)

    # Extracting audio data
    wav, sr = librosa.load(audio_filename, sr=SAMPLE_RATE)
    bg1_wav,sr =librosa.load(bg1_file, sr=SAMPLE_RATE)
    bg2_wav,sr =librosa.load(bg2_file, sr=SAMPLE_RATE)

    # Randomising and normalising audio data
    wav = round_up_audio(wav)
    wav /= np.max(np.abs(wav), axis=0)
    bg1_wav /= np.max(np.abs(bg1_wav), axis=0)
    bg2_wav /= np.max(np.abs(bg2_wav), axis=0)

    # Getting a random audio and bg second
    rand_audio_sec = random_sec(wav)
    bg1_random_wav = random_sec(bg1_wav)
    bg2_random_wav = random_sec(bg2_wav)
    
    # Adding echo and bg noise to the audio
    echo_audio = add_echo_from_file(rir_file, rand_audio_sec)
    #print(echo_audio.shape, bg_wav.shape)
    bg_random_wav = bg1_random_wav+bg2_random_wav
    noisy_audio, noise_lab = add_noise(echo_audio, bg_random_wav)

    noisy_audio = torch.from_numpy(noisy_audio).unsqueeze(0)
    noise_label = torch.from_numpy(noise_lab).unsqueeze(0)

    return noisy_audio, noise_label


In [198]:
spectro_fn = T.Spectrogram(power=None)
inv_spectro_fn = T.InverseSpectrogram()

In [199]:
class audioDataset(Dataset):

    def __init__(self, speech_list):
        #self.audio_df = pd.read_csv(audio_csvfile)
        self.speech_list = speech_list
        #self.audio_dir = audio_dir

    def __len__(self):
        return len(self.speech_list)
    
    def __getitem__(self, index):
    
        audio_path = self.speech_list[index]
        audio, label = get_random_audio_sec(audio_path)

        audio_spec = spectro_fn(audio)
        label_spec = spectro_fn(label)

        return audio_spec.real, label_spec.real

In [200]:
train_dataset = audioDataset(speech_)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [202]:
torch.manual_seed(13)
torch.cuda.manual_seed(13)

class SpecAE(nn.Module):

    def __init__(self):
        super(SpecAE, self).__init__()
        
        self.enc = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(64, 32, kernel_size=3, stride=2,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(32, 16, kernel_size=3, stride=2,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=1,padding=1),
            #nn.LeakyReLU()
            )

        self.dec = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(16, 32, kernel_size=3, stride=2,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(32, 64, kernel_size=3, stride=2,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=3, stride=1,padding=1),
            nn.LeakyReLU(),
    
            nn.Sigmoid()
            )
    
    def forward(self, x):
        x = self.enc(x)
        x = self.dec(x)

        return x

In [203]:
torch.manual_seed(13)
torch.cuda.manual_seed(13)

model = SpecAE()
#model.load_state_dict(torch.load('specAEmodel01.pt'))
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

model = model.to(device)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [204]:
import statistics

epochs = 10

for epoch in range(0,epochs):
    loss_list = []
    for data in train_dataloader:
        
        model.train()
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = loss_fn(outputs, inputs)

        # BP and optim
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())

    print(f"Epoch [{epoch + 1}/{epochs}] Loss: {statistics.mean(loss_list)}")

RuntimeError: Input type (double) and bias type (float) should be the same